## Model Selection: Multiple Linear Regression and Principal Component Analysis

*By Juan Vega*

The goal of this notebook is to fit a multiple linear regression notebook and evaluate its ability to predict the death rate per capita per 100,000 population. Principal Component Analysis will be used to understand which features have the most explanatory power and perform dimensionality reduction.

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

### Ingest the data source of COVID-19 death rate and additional features from the CDC and other data sources

In [ ]:
covid_df = pd.read_csv('./data/merged_dataset.csv')
covid_df.head()

In [ ]:
covid_df.shape

### Rename certain features

In [ ]:
covid_df.rename(columns={'total_staffed_adult_icu_beds_7_day_sum':'tot_staff_adlt_icu_beds',
                        'total_icu_beds_7_day_sum':'total_icu_beds',
                        'inpatient_beds_7_day_sum':'inpatient_beds',
                        'total_adult_patients_hospitalized_confirmed_covid_7_day_sum':'tot_adlt_pat_hosp_covid',
                        'inpatient_beds_used_7_day_sum':'inpatient_beds_used',
                        'total_beds_7_day_sum':'total_beds',
                        'icu_beds_used_7_day_sum':'icu_beds_used'},inplace=True)

In [ ]:
features = [col for col in covid_df.columns if covid_df[col].dtypes != 'O']

In [ ]:
for col in ['county_fips_code','deaths','deaths_per_hun_thou']:
    features.remove(col)

#### Impute missing data using a median imputer for every feature with missing data

In [ ]:
covid_df.isnull().sum()[covid_df.isnull().sum()>0].sort_values(ascending=False)

In [ ]:
df = covid_df[features+['deaths_per_hun_thou']]

In [ ]:
X = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(df.drop(columns='deaths_per_hun_thou')),
                  columns=df.drop(columns='deaths_per_hun_thou').columns)
y = df['deaths_per_hun_thou']

After imputing all missing data with the median of each feature, there are no more missing values in the data set.

In [ ]:
X.isnull().sum().sum(), y.isnull().sum().sum()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=43)

Standardize the features for PCA and multiple linear regression.

In [ ]:
ss = StandardScaler()
Xs_train = pd.DataFrame(ss.fit_transform(X_train),columns=X_train.columns)
Xs_test = pd.DataFrame(ss.transform(X_test),columns=X_test.columns)

Instantiate a linear regression model.

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(Xs_train,y_train)

In [ ]:
y_preds = lr.predict(Xs_test)

Using these features does not produce an R-squared score that is appropriate to model on and compare the performance of these model against the null model.

In [ ]:
y_train_null = [np.mean(y_train)] * len(y_train)
y_test_null = [np.mean(y_train)] * len(y_test)

Although the model fit performs better than the null model based on the r-squared metric, the performance of this model is very poor.

In [ ]:
r2_score(y_test,y_preds)

In [ ]:
r2_score(y_test_null,y_preds)

In [ ]:
plt.figure(figsize=(5,10))
plt.title('Correlation of features with death rate')
sns.heatmap(df.corr()[['deaths_per_hun_thou']].sort_values(by='deaths_per_hun_thou',ascending=False),
                vmin=-1,
                vmax=1,
                cmap='Blues',
                annot=True);

Attempt fitting a better model using only the featues that are mostly correlated with the death rate based on the correlation heatmap above.

In [ ]:
Xs_train = Xs_train[['cases_per_10k','SVI__D','transmission_level_high','med_cases_per_100k_change',
'mask_sometimes','mask_never','med_per_test_results_positive','good_masking_practices','tot_pop']]

Xs_test = Xs_test[['cases_per_10k','SVI__D','transmission_level_high','med_cases_per_100k_change',
'mask_sometimes','mask_never','med_per_test_results_positive','good_masking_practices','tot_pop']]

In [ ]:
lr.fit(Xs_train,y_train)

In [ ]:
y_preds = lr.predict(Xs_test)

In [ ]:
r2_score(y_test,y_preds)

This is an improvement from the current model's performance, demonstrating that a lower number of features may return better results. However, the performance of this model is not adequate at the moment, which is the reason why a collective decision made to fit a classification model for whether the median number of deaths is above the 75th percentile across all U.S. counties.

As an exercise, apply principal component analysis to identify which features explain most of the variation in the data in this model.

In [ ]:
X = X[['cases_per_10k','SVI__D','transmission_level_high','med_cases_per_100k_change',
'mask_sometimes','mask_never','med_per_test_results_positive','good_masking_practices','tot_pop']]

In [ ]:
Xs = ss.fit_transform(X)

In [ ]:
pca = PCA().fit(Xs)

It appears that about 82% of the variation in the data can be explained by the first principal components, which are described below.

In [ ]:
sum(pca.explained_variance_ratio_[:5])

In [ ]:
pca.explained_variance_ratio_

* The first principal component explains around 30% of the variation in the data and appears to describe counties where a higher number of cases per capita is associated with a lower death rate, where a higher level of social vulnerability is associated with a lower death rates, and where higher transmissibility is assoficated with lower death rates, which are counterintuitive results
* The second principal compoennt appears to describe cases where higher number of COVID cases relate to higher death rates, where higher social vulnerability is associated with higher death rates, and higher transmission is associated with higher death rates. This principal component accounts for 20% of the variation in the data.

In [ ]:
pca1_evec = pca.components_[0]

for weight, feature in zip(pca1_evec, X.columns):
    print(feature, weight)
    
# This code is inspired from Sophie's lesson on PCA:
# https://git.generalassemb.ly/dsi-andromeda/pca/blob/master/starter-code-Sophie.ipynb

In [ ]:
pca1_evec = pca.components_[1]

for weight, feature in zip(pca1_evec, X.columns):
    print(feature, weight)

In [ ]:
covid_df_pca= pd.DataFrame(pca.transform(Xs),columns=['pr_comp_0','pr_comp_1','pr_comp_2','pr_comp_3','pr_comp_4',
                                                      'pr_comp_5','pr_comp_6','pr_comp_7','pr_comp_8'])

In [ ]:
cross_val_score(LinearRegression(),
                covid_df_pca,
                covid_df['deaths_per_hun_thou'],
                cv=5).mean()

In [ ]:
cross_val_score(LinearRegression(),
                covid_df_pca.drop(columns=['pr_comp_5','pr_comp_6','pr_comp_7','pr_comp_8']),
                covid_df['deaths_per_hun_thou'],
                cv=5).mean()

Using the first five principal components from the reduced data set returns a cross validated r-square score of 16%, which is a lower explanatory model compared to a previous linear regression model without PCA.

Because none of these performances are adequate, it is recommended that other predictive tasks such as classifying counties as having a death rate that is above the 75th percentile are applied instead of multiple linear regression.